In [122]:
import pandas as pd
import numpy as np
import openpyxl
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

In [2]:
delay_df = pd.read_excel('data/get_around_delay_analysis.xlsx', sheet_name='rentals_data')
delay_doc_df =  pd.read_excel('data/get_around_delay_analysis.xlsx', sheet_name='Documentation')
pricing_df = pd.read_csv('data/get_around_pricing_project.csv')

In [3]:
delay_df.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


In [4]:
delay_df.describe(include='all')

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


In [5]:
delay_df.dtypes

rental_id                                       int64
car_id                                          int64
checkin_type                                   object
state                                          object
delay_at_checkout_in_minutes                  float64
previous_ended_rental_id                      float64
time_delta_with_previous_rental_in_minutes    float64
dtype: object

In [6]:
#Nombre de nan
total_nan = delay_df.isnull().sum()

#Pourcentage
100 * total_nan / delay_df.shape[0]

rental_id                                      0.000000
car_id                                         0.000000
checkin_type                                   0.000000
state                                          0.000000
delay_at_checkout_in_minutes                  23.294228
previous_ended_rental_id                      91.360863
time_delta_with_previous_rental_in_minutes    91.360863
dtype: float64

In [7]:
# Contenu des colonnes
with pd.option_context('display.max_colwidth', None):
  display(delay_doc_df)

,field name,Comment
0,rental_id,Unique identifier of the rental
1,car_id,Unique identifier of the car
2,checkin_type,"Flow used for both checkin and checkout. (ie. access and return the car)\nmobile = rental agreement signed on the owner's smartphone\nconnect = car equiped with the Connect technology , opened by the driver with his smartphone.\nNote: paper contracts were excluded from the data as we have no data on their delay at checkout and it's negligible use case"
3,state,canceled means that the rental did not happen (was canceled by the driver or the owner).
4,delay_at_checkout_in_minutes,Difference in minutes between the rental end time requested by the driver when booking the car and the actual time the driver completed the checkout. Negative values mean that the driver returned the car in advance.
5,previous_ended_rental_id,id of the previous ended rental of the car (NULL when no previous rental or delay with previous rental higher than 12 hours)
6,time_delta_with_previous_rental_in_minutes,"Difference in minutes between this rental planned start time and the previous rental planned end time (when lower than 12 hours, NULL if higher)"


In [ ]:
# Nombre de canceled
display(delay_df[delay_df['state'] == 'canceled'].shape[0])

# Nombre de ended
display(delay_df[delay_df['state'] == 'ended'].shape[0])

#Nombre de canceled et ended qui ont delay_at_checkout_in_minutes null
display(delay_df[delay_df['delay_at_checkout_in_minutes'].isna()]['state'].value_counts())

3265

18045

state
canceled    3264
ended       1700
Name: count, dtype: int64

On peut voir que sur 3265 lignes canceled, 3264 sont avec un retard null.
Regardons le retard precedent quand c'est canceled

In [74]:
delay_prevRent_df = delay_df[pd.notna(delay_df["previous_ended_rental_id"])]

delay_prevRent_df['previous_ended_rental_id'] = [int(x) for x in delay_prevRent_df['previous_ended_rental_id']]

delay_prevRent_df['previous_delay_at_checkout_in_minutes'] = [
    delay_df[delay_df['rental_id'] == prev_car].delay_at_checkout_in_minutes.values[0]
    for prev_car in delay_prevRent_df['previous_ended_rental_id']]


delay_canceled_df = delay_prevRent_df[delay_df['state'] == 'canceled'].reset_index(drop=True)
delay_canceled_df
# Il reste que 229 lignes

C:\Users\DELL\AppData\Local\Temp\ipykernel_22156\1552696507.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DELL\AppData\Local\Temp\ipykernel_22156\1552696507.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DELL\AppData\Local\Temp\ipykernel_22156\1552696507.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_delay_interval_minutes,previous_delay_at_checkout_in_minutes
0,543768,374169,connect,canceled,NaN,543010,210.0,NA,42.0
1,546160,352528,connect,canceled,NaN,546578,630.0,NA,-536.0
2,564627,341431,mobile,canceled,NaN,552005,150.0,NA,-88.0
3,568657,317378,connect,canceled,NaN,566412,210.0,NA,-129.0
4,516550,377700,mobile,canceled,NaN,545076,720.0,NA,-35.0
...,...,...,...,...,...,...,...,...,...
224,566228,390871,connect,canceled,NaN,568465,60.0,NA,NaN
225,567305,405564,mobile,canceled,NaN,552222,0.0,NA,NaN
226,569706,245154,connect,canceled,NaN,558088,660.0,NA,-40.0
227,568049,381499,connect,canceled,NaN,562174,720.0,NA,73.0


In [79]:
px.histogram(delay_canceled_df, 'previous_delay_at_checkout_in_minutes', barmode="group", color='checkin_type', nbins=int(np.sqrt(delay_canceled_df.shape[0])))

EDA

In [91]:
fig = px.box(delay_df, x='delay_at_checkout_in_minutes', color='checkin_type')
fig.show()

In [35]:
fig = px.scatter(delay_df, x = 'delay_at_checkout_in_minutes')
fig.show()

In [36]:
def get_interval(delay: int):
    intervals = [
        (float('-inf'), 0, 'early'),
        (0, 15, 'slightly_late'),
        (15, 30, 'moderately_late'),
        (30, 60, 'late'),
        (60, 120, 'very_late'),
        (120, 1440, 'extremely_late'),
        (1440, float('inf'), 'severely_late')
    ]

    for lower, upper, category in intervals:
        if lower < delay <= upper:
            return category

    return 'NA'

delay_df['checkout_delay_interval_minutes'] = delay_df['delay_at_checkout_in_minutes'].apply(lambda x: get_interval(x))
delay_df

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_delay_interval_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN,NA
1,507750,269550,mobile,ended,-81.0,NaN,NaN,early
2,508131,359049,connect,ended,70.0,NaN,NaN,very_late
3,508865,299063,connect,canceled,NaN,NaN,NaN,NA
4,511440,313932,mobile,ended,NaN,NaN,NaN,NA
...,...,...,...,...,...,...,...,...
21305,573446,380069,mobile,ended,NaN,573429.0,300.0,NA
21306,573790,341965,mobile,ended,-337.0,NaN,NaN,early
21307,573791,364890,mobile,ended,144.0,NaN,NaN,extremely_late
21308,574852,362531,connect,ended,-76.0,NaN,NaN,early


In [37]:
type_checkout = delay_df['checkout_delay_interval_minutes'].value_counts().reset_index()
type_checkout.columns = ['checkout_delay_interval_minutes', 'count']
type_checkout

,checkout_delay_interval_minutes,count
0,early,6942
1,NA,4964
2,extremely_late,2365
3,slightly_late,1988
4,very_late,1833
5,late,1692
6,moderately_late,1338
7,severely_late,188


In [ ]:
fig = px.bar(type_checkout, x='checkout_delay_interval_minutes', y='count')
fig.show()

In [83]:
delay_df.shape[0]
print(np.sqrt(delay_df.shape[0]))

145.97945060863876


In [ ]:
px.histogram(delay_df, 'delay_at_checkout_in_minutes', barmode="group", color='checkin_type', nbins=int(np.sqrt(delay_df.shape[0])))

Distribution normal mais avec des outliers

Trop de valeur, graphique pas representatif

Suppression des outilers

In [87]:
def remove_outliers(data, lower_quantile: float = 0.01, upper_quantile: float = 0.99) -> pd.Series:

    # Calculate the quantiles
    lower_bound = data['delay_at_checkout_in_minutes'].quantile(lower_quantile)
    upper_bound = data['delay_at_checkout_in_minutes'].quantile(upper_quantile)

    # Filter the data to remove outliers
    filtered_data = data[(data['delay_at_checkout_in_minutes'] >= lower_bound) & (data['delay_at_checkout_in_minutes'] <= upper_bound)]

    return filtered_data.reset_index()

delay_clean_df = remove_outliers(delay_df)
delay_clean_df.describe(include='all')

,index,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_delay_interval_minutes
count,16019.000000,16019.000000,16019.000000,16019,16019,16019.000000,1483.000000,1483.000000,16019
unique,NaN,NaN,NaN,2,1,NaN,NaN,NaN,7
top,NaN,NaN,NaN,mobile,ended,NaN,NaN,NaN,early
freq,NaN,NaN,NaN,12657,16019,NaN,NaN,NaN,6779
mean,10683.828828,549866.584244,349015.453898,NaN,NaN,31.599600,549914.071477,276.898179,NaN
std,6152.952751,13587.091818,58230.502885,NaN,NaN,215.203776,13421.152503,254.294201,NaN
min,1.000000,504806.000000,159250.000000,NaN,NaN,-853.000000,505628.000000,0.000000,NaN
25%,5349.500000,540898.500000,316968.000000,NaN,NaN,-35.000000,540598.500000,60.000000,NaN
50%,10663.000000,550402.000000,367531.000000,NaN,NaN,9.000000,550539.000000,180.000000,NaN
75%,16028.500000,560255.000000,393639.500000,NaN,NaN,65.000000,560812.000000,540.000000,NaN


In [88]:
px.histogram(delay_clean_df, 'delay_at_checkout_in_minutes', barmode="group", color='checkin_type', nbins=int(np.sqrt(delay_clean_df.shape[0])))

Nombre de retard pour les locations avec un délai de - 12h d'ecart

In [ ]:
valid_df = delay_df.dropna(subset=["time_delta_with_previous_rental_in_minutes"])

problematic_df = valid_df[(valid_df["delay_at_checkout_in_minutes"] > 0) &
                       (valid_df["time_delta_with_previous_rental_in_minutes"] < valid_df["delay_at_checkout_in_minutes"])]

print("Nombre de cas problématiques :", problematic_df.shape[0])

Nombre de cas problématiques : 270


Retard en fonction du délai entre 2 locations

In [96]:
fig = px.scatter(valid_df, x='time_delta_with_previous_rental_in_minutes', y='delay_at_checkout_in_minutes')
fig.show()

## Pricing

In [98]:
pricing_df = pd.read_csv('data/get_around_pricing_project.csv')
pricing_df

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,4838,Toyota,39743,110,diesel,black,van,False,True,False,False,False,False,True,121
4839,4839,Toyota,49832,100,diesel,grey,van,False,True,False,False,False,False,True,132
4840,4840,Toyota,19633,110,diesel,grey,van,False,True,False,False,False,False,True,130
4841,4841,Toyota,27920,110,diesel,brown,van,True,True,False,False,False,False,True,151


In [99]:
pricing_df.describe(include="all")

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843.000000,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,NaN,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,NaN,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,NaN,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,2421.000000,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,1398.198007,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,0.000000,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,1210.500000,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,2421.000000,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,3631.500000,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


In [ ]:
# Missing values
100 * pricing_df.isnull().sum() / pricing_df.shape[0]

# Pas de missing values

Unnamed: 0                   0.0
model_key                    0.0
mileage                      0.0
engine_power                 0.0
fuel                         0.0
paint_color                  0.0
car_type                     0.0
private_parking_available    0.0
has_gps                      0.0
has_air_conditioning         0.0
automatic_car                0.0
has_getaround_connect        0.0
has_speed_regulator          0.0
winter_tires                 0.0
rental_price_per_day         0.0
dtype: float64

In [ ]:
fig = px.histogram(pricing_df, "rental_price_per_day")

outliers_min = np.mean(pricing_df['rental_price_per_day']) - 3 * np.std(pricing_df['rental_price_per_day'])
outliers_max = np.mean(pricing_df['rental_price_per_day']) + 3 * np.std(pricing_df['rental_price_per_day'])

display('Min : ' + str(outliers_min), 'Max : ' + str(outliers_max))

fig.add_vline(x=outliers_min, line_dash = 'dash', line_color = 'red')
fig.add_vline(x=outliers_max, line_dash = 'dash', line_color = 'red')
fig.show()

'Min : 20.520130521253677'

'Max : 221.9089423674517'

In [ ]:
pricing_df[(pricing_df['rental_price_per_day'] > 20) & (pricing_df['rental_price_per_day'] < 222)]

# 54 lignes, 11% des lignes

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183
5,5,Citroën,152352,225,petrol,black,convertible,True,True,False,False,True,True,True,131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,4838,Toyota,39743,110,diesel,black,van,False,True,False,False,False,False,True,121
4839,4839,Toyota,49832,100,diesel,grey,van,False,True,False,False,False,False,True,132
4840,4840,Toyota,19633,110,diesel,grey,van,False,True,False,False,False,False,True,130
4841,4841,Toyota,27920,110,diesel,brown,van,True,True,False,False,False,False,True,151


In [ ]:
# Verification de mileage, on peut voir une valeur negative dans le describe (impossible)

display(pricing_df.loc[pricing_df['mileage'] < 0,:])

px.box(pricing_df, "mileage")

# 1 seule valeur négatif, à supprimer pour le model de prediction
# 1 valeur à 1M, à voir pendant l'entrainment si on la garde

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
2938,2938,Renault,-64,230,diesel,black,sedan,True,True,False,True,False,False,True,274


In [118]:
# Valeurs uniques pour chaque colonne spécifiée (qualitatif)

cols = ['model_key', 'mileage', 'engine_power', 'fuel', 'paint_color', 'car_type']

unique_values = {col: pricing_df[col].unique() for col in cols}
unique_values

{'model_key': array(['Citroën', 'Peugeot', 'PGO', 'Renault', 'Audi', 'BMW', 'Ford',
        'Mercedes', 'Opel', 'Porsche', 'Volkswagen', 'KIA Motors',
        'Alfa Romeo', 'Ferrari', 'Fiat', 'Lamborghini', 'Maserati',
        'Lexus', 'Honda', 'Mazda', 'Mini', 'Mitsubishi', 'Nissan', 'SEAT',
        'Subaru', 'Suzuki', 'Toyota', 'Yamaha'], dtype=object),
 'mileage': array([140411,  13929, 183297, ...,  19633,  27920, 195840], shape=(4786,)),
 'engine_power': array([100, 317, 120, 135, 160, 225, 145, 105, 125, 130, 270, 110, 140,
        180,  85, 190, 236, 240, 250, 230, 309, 170, 142, 155, 210, 122,
        200,  90, 150, 141, 173, 137, 280, 112, 189,  80, 119,  95, 220,
        107, 127,  25,  70, 192, 101,  75, 239, 235,  87,  66,  77, 195,
        183, 233, 121, 224, 412,   0, 423, 300, 261]),
 'fuel': array(['diesel', 'petrol', 'hybrid_petrol', 'electro'], dtype=object),
 'paint_color': array(['black', 'grey', 'white', 'red', 'silver', 'blue', 'orange',
        'beige', 'brown', 

In [129]:
cols = ['has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']

fig = make_subplots(rows=len(cols), cols=1, subplot_titles=cols)

for i, col in enumerate(cols):
    hist = px.histogram(pricing_df, x='rental_price_per_day', color=col, barmode="group")
    
    for trace in hist.data:
        fig.add_trace(trace, row=i+1, col=1)

fig.update_layout(height=300 * len(cols), showlegend=False)

fig.show()